In [1]:
import requests
import sqlite3
from sqlite3 import Error
import json

This notebook is for reading the sersver trip and matching the relevant user input with the phone UserCache's IMU data for each segment.

1. Connects to an instance of the e-mission server
2. Accepts a user UUID (e.g. email) and use it to fetch all processed trip data
3. For each trip segment, fetch all corresponding IMU data from the SQLite phone UserCache local database
4. Also for each trip segment, check and fetch user input ground truth from the UserCache.

For more details refer to https://github.com/e-mission/e-mission-docs/issues/627

In [2]:
# Replace these global variables with your actual setup.
SERVER_URL = "http://45.33.108.175:80"
LOOKUP_DATE = "2021-03-01"
USERNAME = "josh01"

post_msg = {
    "user": USERNAME
}

# To obtain the local database, use E-mission -> Profile -> Developer zone -> Check sensed data.
# For more details, check https://github.com/e-mission/e-mission-docs/issues/613
PHONE_LOCAL_DATABASE_LOCATION = "/Users/bowenxu2/Downloads/userCacheDB"

In [3]:
# Helper method to create the phone local database connection.

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

def select_last_row(conn, timestamp_key):
    """
    Query all rows in the userCache table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    cur.execute("SELECT * FROM userCache WHERE key='" + timestamp_key + "' LIMIT 5")

    rows = cur.fetchall()

    print("The number of user input results for the current segment motion mode:", len(rows))
#     for row in rows:    
#         print(row)
    if len(rows) > 0:
        last_row = rows[-1]
        print("last row:", last_row)
        print("latest user input:", json.loads(last_row[6])["label"])
    else:
        print("no user input for current segment")

In [4]:
# Load the UserCache (SQLite database) and create the connection
conn = create_connection(PHONE_LOCAL_DATABASE_LOCATION)

In [5]:
# Get the E-mission server response.
# The E-mission server uses a MongoDB (NoSQL), so there's no direct way of obtaining the dump containing all user data on all dates.
# We need to use the POST API to obtain the daily data once at a time.
response = requests.post(SERVER_URL + "/timeline/getTrips/" + LOOKUP_DATE, json=post_msg)

In [6]:
today_trip_list = response.json()['timeline']
print("today's number of trips:", len(today_trip_list))

today's number of trips: 6


In [7]:
for idx, trip in enumerate(today_trip_list):
    print("Reviewing trip", idx + 1, "of", len(today_trip_list))
    print("trip start time:", trip['properties']['start_fmt_time'])
    print("trip end time:", trip['properties']['end_fmt_time'])
    print("Reading segment data...")
    segment_list = []
    for feature in trip['features']:
        if feature['type'] == "FeatureCollection":
            segment_list.append(feature)
            segment_start_timestamp = str(feature['features'][0]['properties']['start_ts'])
            segment_end_timestamp = str(feature['features'][0]['properties']['end_ts'])
            print("current segment start time:", feature['features'][0]['properties']['start_fmt_time'], segment_start_timestamp)
            print("current segment end time:", feature['features'][0]['properties']['end_fmt_time'], segment_end_timestamp)
            raw_motion_mode_text = feature['features'][0]['properties']['sensed_mode']
            print("current segment motion mode, detected by E-mission model:", raw_motion_mode_text.split(".")[1])
            # To find the matching user input in the local database, get the key by "StartTimeStamp_EndTimeStamp"
            select_last_row(conn, segment_start_timestamp + "_" + segment_end_timestamp)
            print("--")
    print("Number of segments in this trip:", len(segment_list))
    print("-" * 20)

Reviewing trip 1 of 6
trip start time: 2021-03-01T10:33:10-08:00
trip end time: 2021-03-01T10:59:50-08:00
Reading segment data...
current segment start time: 2021-03-01T10:33:10-08:00 1614623590.0
current segment end time: 2021-03-01T10:59:50-08:00 1614625190.0
current segment motion mode, detected by E-mission model: CAR
The number of user input results for the current segment motion mode: 0
no user input for current segment
--
Number of segments in this trip: 1
--------------------
Reviewing trip 2 of 6
trip start time: 2021-03-01T11:09:44.717288-08:00
trip end time: 2021-03-01T11:47:13.945000-08:00
Reading segment data...
current segment start time: 2021-03-01T11:09:44.717288-08:00 1614625784.717288
current segment end time: 2021-03-01T11:47:13.945000-08:00 1614628033.945
current segment motion mode, detected by E-mission model: BICYCLING
The number of user input results for the current segment motion mode: 2
last row: (1615506968.765, None, 'America/Los_Angeles', 'message', '161462